# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Zadanie domowe 2

In [ ]:
import pandas as pd
import numpy as np
       

## Zadanie 1 - eksploracja przestrzeni zagnieżdżeń
Wczytajmy do przestrzeni plik zagnieżdżeń, który należy pobrać ze strony:
https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec Są to zagnieżdzenia dla języka polskiego uzyskane systemem fastText.

Do przestrzeni wczytujemy tylko 100 tys. najczęstrzych słów, tak aby operacje przebiegały szybciej.

In [ ]:
import io
def load_vectors(fname, limit = 100000):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())
    n = min(n,limit)
    embeddings = np.empty((n,d), dtype = np.float)
    words_idx = []
    for i, line in enumerate(fin):
        if i >= limit:
            break
        tokens = line.rstrip().split(' ')
        words_idx.append(tokens[0])
        embeddings[i] =  np.array(tokens[1:]).astype(np.float)
    return words_idx, embeddings
words_idx, embeddings = load_vectors('wiki.pl.vec')

Poniższe zadania mają na celu poekserymentowanie z przestrzenią zagnieżdżeń, ale też zrozumienie stojącymi za nich operacji. Dozwolone jest korzystanie tylko z podstawowych operatorów Python i numpy (w szczególności zakaz dotyczy: sklearn, gensim, fasttext itd.)

Jeśli potrzebujesz do dalszego przetwarzania przeprowadzenia jakichś normalizacji macierzy -- możesz wstępnie przetworzyć macierz zagnieżdzeń poniżej. Pamiętaj, że sprawdzarka będzie używała wywołań do `embeddings` (i `words_idx`) -- musisz nadpisać macierz zagnieżdzeń. To pole jest pomocnicze i nie podlega ocenie.

In [ ]:
# YOUR CODE HERE
# print(embeddings.shape)
# print(embeddings[words_idx.index("dom")].shape)
embeddings = embeddings * (1 / np.linalg.norm(embeddings, axis=1))[:, np.newaxis]
# print(embeddings.shape)
# print(embeddings[words_idx.index("dom")].shape)
# print()
# print(words_idx.index("ala"))
# print(words_idx.index("ma"))
# print(words_idx.index("kota"))
# print(embeddings[words_idx.index("dom")].shape)

Zaimplementuj funkcję, która obliczy podobieństwo kosinusowe pomiędzy dwoma wyrazami.

In [ ]:
def calc_sim(word_a, word_b, words_idx, embeddings):
    emb_a = embeddings[words_idx.index(word_a)]
    emb_b = embeddings[words_idx.index(word_b)]
    cos_sim = np.dot(emb_a, emb_b)
    return cos_sim

In [ ]:
from nose.tools import assert_almost_equal
assert_almost_equal(calc_sim("bieber", "rihanna", words_idx, embeddings), calc_sim("rihanna", "bieber", words_idx, embeddings))

Policz podobieństwo pomiędzy wyrazem `bieber` a wyrazami:
    - `rihanna`
    - `piłsudski`
    - `kanada`
    - `polska`
    - `piosenka`

In [ ]:
calc_sim("bieber", "rihanna", words_idx, embeddings)

In [ ]:
# YOUR CODE HERE
def print_similarity_to_bieber(word):
    cos_sim = calc_sim("bieber", word, words_idx, embeddings)
    print(f"bieber to {word}:\t {str(cos_sim)}")
    
print_similarity_to_bieber("rihanna")
print_similarity_to_bieber("piłsudski")
print_similarity_to_bieber("kanada")
print_similarity_to_bieber("polska")
print_similarity_to_bieber("piosenka")

Zaimplementuj funkcję, która zwróci najbardziej podobne słowa (miara kosinusowa) do danego słowa `word`. W wyniku wypisz tylko `top` słów z najbliższymi zagnieżdżeniami, pomijając słowo `word`.

In [ ]:
def find_similar(word, words_idx, embedding_matrix, top=10):
    cosine_similarities = []
    for word_idx in words_idx:
        similarity = calc_sim(word, word_idx, words_idx, embedding_matrix)
        cosine_similarities.append(similarity)
    top_indicies = np.argsort(np.negative(cosine_similarities))[1:top+1]
    
    result_arr = []
    for top_id in top_indicies:
        result_arr.append(words_idx[int(top_id)])
    return result_arr

In [ ]:
assert len(find_similar("radość", words_idx, embeddings)) == 10

Znajdź najbardziej podobne słowa do kobieta, politechnika, mateusz, szczecin, niemcy, piłsudski

In [ ]:
find_similar("kobieta", words_idx, embeddings)


In [ ]:
# YOUR CODE HERE
arr = ["kobieta", "politechnika", "mateusz", "szczecin", "niemcy", "piłsudski"]
for word in arr:
    print(word)
    print(find_similar(word, words_idx, embeddings))
    

Krótko skomentuj wyniki dla słowa `niemcy`. Które z powstałych analogii biorą się z semantycznego powiązania a które z semantycznego podobieństwa?

In [ ]:
# YOUR CODE HERE
# powiazanie: niemieccy, niemieckie niemców - słowa te współwystepują jako
# różne odmiany tego samego słowa ale niosąc to samo znaczenie.

# podobieństwo: naziści, alianci, okupanci, polacy, itd - słowa te 
# występują na poodbnych pozycjach w zdaniu

Zaimplementuj funkcje szukającą brakującego elementu relacji ,,`word_a` jest do `word_a2` jak `word_b` jest do...''. Funkcja powinna zwrócić 10 najbardziej pasujących słow z pominięciem słów będących jej argumentami.

In [ ]:
def find_similar_pair(word_a, word_a2, word_b,  words_idx, matrix, top=10):
    # YOUR CODE HERE
    emb_a = matrix[words_idx.index(word_a)]
    emb_a_2 = matrix[words_idx.index(word_a2)]
    emb_b = matrix[words_idx.index(word_b)]
    diff_a = emb_a - emb_a_2
    emb_b_2 = emb_b - diff_a
    
    cosine_similarities = []
    for word_idx in words_idx:
        if word_idx != word_a and word_idx != word_a2 and word_idx != word_b:
            similarity = np.dot(emb_b_2, embeddings[words_idx.index(word_idx)])
        else:
            similarity = 0
        cosine_similarities.append(similarity)
    top_indicies = np.argsort(np.negative(cosine_similarities))[:top]
    
    result_arr = []
    for top_id in top_indicies:
        result_arr.append(words_idx[int(top_id)])
    return result_arr
    
    

In [ ]:
assert find_similar_pair( "mężczyzna", "król", "kobieta", words_idx, embeddings)[0] == "królowa"

Pieniądze są do profesora jak wiedza do...

In [ ]:
# YOUR CODE HERE
find_similar_pair("pieniądze", "profesora", "wiedza", words_idx, embeddings)

Mateusza jest do mateusz jak łukasza do ...

In [ ]:
# YOUR CODE HERE
find_similar_pair("mateusza", "mateusz", "łukasza", words_idx, embeddings)

Warszawa jest do "polska" jak "berlin" do ...

In [ ]:
# YOUR CODE HERE
find_similar_pair("warszawa", "polska", "berlin", words_idx, embeddings)

Zurich jest do ETH jak Poznań do ...

In [ ]:
find_similar_pair("zurich", "eth", "poznań", words_idx, embeddings)

Niemcy są do Merkel jak Polska do ...

In [ ]:
find_similar_pair( "niemcy", "merkel", "polska", words_idx, embeddings)

Na wektorach możemy wykonywać standardowe operacje algebry liniowej takie jak np. projekcja czyli rzutowanie danych na jakichś zbiór osi (więcej: notatki z algebry liniowej np. https://ocw.mit.edu/courses/mathematics/18-06sc-linear-algebra-fall-2011/least-squares-determinants-and-eigenvalues/projections-onto-subspaces/). W szczególności może to się przydać do zrzutowania słowa na przestrzeń w której pewny wybrany kierunek (wskazywany przez wektor) jest eliminowany.

Do czego może to się przydać? Jeśli uruchomisz funkcję `find_similar` dla słowa ,,mateusza'' znajdziesz m.in. ,,łukasza'' ale także ,,ewangelia'', ,,ewangelisty'' i ,,apostoła''. Chcąc pominąc kontekst religijny tego słowa możesz zrzutować jego reprezentacje na przestrzeń bez wektora ,,ewangelia'' i poszukać jego najbliższych sąsiadów (którymi będą teraz po prostu imiona męskie). Zaimplementuj taką funkcję.


In [ ]:
def find_similar_with_rejection(word, remove, words_idx, matrix, top=10):
    """
    Działanie analogiczne do find_similar z dodatkowym parametrem remove, 
    który jest *listą* słów, które należy wyrzucić poprzez projekcję.
    Dla remove=[] powinno się zwracać dokładnie to samo co find_similar
    """
    # YOUR CODE HERE
    return find_similar(word, words_idx, matrix, top)
print ("Standardowe poszukiwanie:", find_similar_with_rejection("mateusza",[] , words_idx, embeddings))
print ("Poszukiwanie po projekcji:", find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings))

In [ ]:
assert "ewangelii" in find_similar_with_rejection("mateusza",[] , words_idx, embeddings)
assert "ewangelii" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)
assert "ewangelisty" not in find_similar_with_rejection("mateusza",["ewangelia"] , words_idx, embeddings)


Analogicznie słowo ,,java'' jest nie tylko nazwą języka programownia (https://pl.wikipedia.org/wiki/Java_(ujednoznacznienie)) -- jest np. nazwą geograficzną (indonezyjska wyspa koło Sumatry). Sprawdź jakie wyrazy są podobne do "java" oraz po odrzuceniu kierunku "javascript" (tj. kierunku związanego z językami programowania).

In [ ]:
# YOUR CODE HERE
# nie zrobiłem :(

Spróbuj poekseprymentować samemu!

In [ ]:
# YOUR CODE HERE
# nie zrobiłem :(

Wykonanie projekcji w przestrzeni zagnieżdżeń może być jedną z prostych technik zwalczenia tzw. gender bias (http://wordbias.umiacs.umd.edu/) w reprezentacji słów. Okazuje się, że wykonanie projekcji macierzy zagnieżdżeń na przestrzeń w której ,,brakuje kierunku he-she'' może być bardzo prostą techniką zredukowania tego typu obciążenia.

## Zadanie 2 - zagnieżdżenia dokumentów
W tym ćwiczeniu powócimy do zbioru tweetów, który analizowaliśmy w poprzednim dokumencie.

In [ ]:
from helpers import DataSet
training_set = DataSet(['tweets.txt'])

In [ ]:
for i in training_set.tweets:
    print(i.text)
    print(i.tokens)
    print(i.clazz)
    break

Tym razem do zbudowania reprezentacji będziemy używać narzędzie Universal Sentence Encoder stworzone przez Googla na bazie głębokiej sieci uśredniającej (i architektur rekurencyjnych). Poniższy kod pokazuje sposób użycia tego narzędzia. 
Kod spokojnie można wywoływać na CPU -- choć ściąganie modelu trochę może potrwać.

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])
print (embeddings)

Wykorzystując reprezetnację USE wytrenuj wybrany klasyfikator z pakietu `sklearn` i zweryfikuj jego jakość działania.

In [ ]:
# YOUR CODE HERE
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
import numpy as np

tweet_text = [tweet.text for tweet in training_set.tweets]
tweet_emb = embed(tweet_text).numpy()

tweet_clazzes = [tweet.clazz for tweet in training_set.tweets]

def fit_classifier_use(tokens, clazzes, no_features):
    X_train, X_test, y_train, y_test = train_test_split(
        tokens, clazzes, test_size=0.1, random_state=42)
    
    text_clf = SGDClassifier(loss='hinge', penalty='l2',
                        alpha=1e-3, random_state=42,
                        max_iter=5, tol=None)
    text_clf.fit(X_train, y_train)
    
    predicted = text_clf.predict(X_test)
    
    # return: accuracy
    return np.mean(predicted == y_test)

fit_classifier_use(tweet_emb, tweet_clazzes, 10)

Skomentuj wyniki i odnieś je do wyników z poprzedniego zadania domowego. Na ile użycie reprezentacji rozproszonych pozwoliło na poprawę wyników?

In [ ]:
# YOUR CODE HERE
# W 3 przypadkach użyto tego samego klasyfikatora, jedyną różnicą 
# (jak się okazało bardzo znaczącą) byłą reprezentacja wejścia.

# 1) wykorzystujący hashowanie cech
#    acc: 0.43 ~ 0.6 (w zależności od parametru wymiaru macierzy danych)

# 2) wykorzytujący Bag-of-clusters
#    acc: 0.57

# 3) wykorzytujący USE embeddings
#    acc: 0.64

# Wnioski: USE mocno poprawiły wynik, jednak w tym przypadku aby osiągnąć
#          jeszcze lepsze wyniki należało by użyć bardziej zaawansowanego 
#          klasyfikatora lub innych technik. Jednakże widać dużą przewgę 
#          USE nad pozostałymi reprezentacjami.

# Zadanie 3 - konstruowanie zagnieżdżeń
W tym ćwiczeniu kontynuujemy pracę z tweetami, ale pomijamy całkowicie ich klasy. Zbiór tweetów potraktujemy jako korpus do nauczenia zagnieżdżeń słów przy pomocy macierzy PMI.
- Wypełnij macierz kontekst - dokument przy użyciu symetrycznego okna o promieniu 4 (po 4 słowa w każdą stronę)
- Możesz ograniczyć słownictwo do 10K słów
- Przekształć macierz w macierz PPMI
- Stwórz zagnieżdżenia wykorzystując dekompozycję SVD do wybranej wymiarowości $d$ (ze względu na koszt obliczeniowy może to być mała wymiarowość np. $d=10$)

In [ ]:
# YOUR CODE HERE
from tensorflow.keras.preprocessing.text import Tokenizer

RADIUS = 4

tweet_texts = [tweet.text for tweet in training_set.tweets]
test_texts = [
    "I like Information Retrieval and I like Statistics.",
    "I enjoy flying."
]

tokenizer = Tokenizer()

def create_hal_matrix(texts):
    
    tokenizer.fit_on_texts(texts)

    size = len(tokenizer.word_index)
    # print(tokenizer.texts_to_sequences(["sth the a an"]))

    hal_matrix = np.zeros((size, size))

    sentences_sequences = tokenizer.texts_to_sequences(texts)

    for sentences_sequence in sentences_sequences:
#         print(sentences_sequence)
        for i, sequence in enumerate(sentences_sequence):
#             print(sequence, i)
            for n in range(1, RADIUS + 1):
                index_prev = i - n
                index_post = i + n
                value = RADIUS + 1 - n
                if index_prev >= 0:
                    hal_matrix[sequence - 1, sentences_sequence[index_prev] - 1] += value
#                     print("PREV", sentences_sequence[index_prev], RADIUS + 1 - n)
                if index_post < len(sentences_sequence):
                    hal_matrix[sequence - 1, sentences_sequence[index_post] - 1] += value
#                     print("POST", sentences_sequence[index_post], RADIUS + 1 - n)
#     print(hal_matrix)
    return hal_matrix

def matrix_to_ppmi(matrix):
    ppmi_matrix = np.zeros_like(matrix, dtype='float64')
    matrix_sum = np.sum(matrix)
    matrix_row_sum = np.sum(matrix, axis = 1)
    matrix_col_sum = np.sum(matrix, axis = 0)
    

    for index, x in np.ndenumerate(matrix):
        ppmi_matrix[index] = (matrix[index] * matrix_sum) / (matrix_row_sum[index[0]] * matrix_col_sum[index[1]])
    
    ppmi_matrix = np.log2(ppmi_matrix)
    ppmi_matrix = np.maximum(ppmi_matrix, 0)
#     print(ppmi_matrix)
    
    return ppmi_matrix

hal_matrix = create_hal_matrix(tweet_texts)
# hal_matrix = np.array([
#     [0, 0, 1, 0, 1],
#     [0, 0, 1, 0, 1],
#     [2, 1, 0, 1, 0],
#     [1, 6, 0, 4, 0],
#     [0, 0, 0, 0, 0]
# ])
hal_matrix = hal_matrix.astype('float64')
# print(hal_matrix)

ppmi_matrix = matrix_to_ppmi(hal_matrix)
# print(ppmi_matrix)

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=10, n_iter=8, random_state=42)
svd.fit(ppmi_matrix)

my_embeddings = svd.transform(ppmi_matrix)

# print(hal_matrix)
# print(ppmi_matrix)

Przetestuj działanie Twoich zagnieżdżeń wykorzystując funkcję `find_similar` na wybranych słowach.

In [ ]:
# YOUR CODE HERE
# YOUR CODE HERE
my_embeddings = my_embeddings * (1 / np.linalg.norm(my_embeddings, axis=1))[:, np.newaxis]
arr = ["good", "ok", "trump", "dog"]
for word in arr:
    print(word)
    print(find_similar(word, list(tokenizer.word_index.keys()), my_embeddings))
